In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [17]:
customer = pd.read_csv('Customers.csv')

In [18]:
products = pd.read_csv('Products.csv')

In [19]:
transactions = pd.read_csv('Transactions.csv')

In [20]:
merged_csv = pd.merge(customer,transactions, on='CustomerID', how='inner')
product_selected = products[['ProductID','Category','ProductName']]
final_csv = pd.merge(merged_csv, product_selected, on='ProductID', how = 'inner')

In [21]:
singup_year = pd.to_datetime(final_csv['SignupDate'])
final_csv['SingupYear'] = singup_year.dt.year

In [22]:
final_csv['TransactionDate'] = pd.to_datetime(final_csv['TransactionDate'])

final_csv['Transaction_Hour'] = final_csv['TransactionDate'].dt.hour

# def categorize_time(hour):
#     if 6 <= hour < 12:
#         return 'Morning'
#     elif 12 <= hour < 18:
#         return 'Afternoon'
#     elif 18 <= hour < 24:
#         return 'Evening'
#     else:
#         return 'Night'

# final_csv['TimePeriod'] = final_csv['Transaction_Hour'].apply(categorize_time)

final_csv

,CustomerID,CustomerName,Region,SignupDate,TransactionID,ProductID,TransactionDate,Quantity,TotalValue,Price,Category,ProductName,SingupYear,Transaction_Hour
0,C0001,Lawrence Carroll,South America,2022-07-10,T00015,P054,2024-01-19 03:12:55,2,114.60,57.30,Books,SoundWave Cookbook,2022,3
1,C0001,Lawrence Carroll,South America,2022-07-10,T00932,P022,2024-09-17 09:01:18,3,412.62,137.54,Home Decor,HomeSense Wall Art,2022,9
2,C0001,Lawrence Carroll,South America,2022-07-10,T00085,P096,2024-04-08 00:01:00,2,614.94,307.47,Electronics,SoundWave Headphones,2022,0
3,C0001,Lawrence Carroll,South America,2022-07-10,T00445,P083,2024-05-07 03:11:44,2,911.44,455.72,Electronics,ActiveWear Smartwatch,2022,3
4,C0001,Lawrence Carroll,South America,2022-07-10,T00436,P029,2024-11-02 17:04:16,3,1300.92,433.64,Electronics,TechPro Headphones,2022,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,C0200,Kelly Cross,Asia,2023-06-11,T00518,P034,2024-12-11 03:05:50,4,868.20,217.05,Clothing,HomeSense T-Shirt,2023,3
996,C0200,Kelly Cross,Asia,2023-06-11,T00091,P057,2024-04-27 19:06:20,1,239.70,239.70,Electronics,ActiveWear Smartphone,2023,19
997,C0200,Kelly Cross,Asia,2023-06-11,T00731,P061,2024-07-15 20:36:28,4,627.84,156.96,Home Decor,HomeSense Desk Lamp,2023,20
998,C0200,Kelly Cross,Asia,2023-06-11,T00771,P048,2024-09-10 09:50:48,4,1665.60,416.40,Books,TechPro Cookbook,2023,9


In [23]:
final_csv.columns

Index(['CustomerID', 'CustomerName', 'Region', 'SignupDate', 'TransactionID',
       'ProductID', 'TransactionDate', 'Quantity', 'TotalValue', 'Price',
       'Category', 'ProductName', 'SingupYear', 'Transaction_Hour'],
      dtype='object')

In [24]:
customer_stats = final_csv.groupby('CustomerID').agg(
    avg_transaction_price=('Price', 'mean'),
    num_transactions=('TransactionID', 'nunique'),
    signup_year=('SingupYear', 'first'),
).reset_index()

region = final_csv['Region']


# favorite_product = final_csv.groupby(['CustomerID', 'ProductID'])['TransactionID'].count().reset_index(name='ProductCount')
# favorite_product = favorite_product.loc[favorite_product.groupby('CustomerID')['ProductCount'].idxmax()][['CustomerID', 'ProductID']]

favorite_category = final_csv.groupby(['CustomerID', 'Category'])['TransactionID'].count().reset_index(name='CategoryCount')
favorite_category = favorite_category.loc[favorite_category.groupby('CustomerID')['CategoryCount'].idxmax()][['CustomerID', 'Category']]

# favorite_time_period = final_csv.groupby(['CustomerID', 'TimePeriod'])['TransactionID'].count().reset_index(name='TimePeriodCount')
# favorite_time_period = favorite_time_period.loc[favorite_time_period.groupby('CustomerID')['TimePeriodCount'].idxmax()][['CustomerID', 'TimePeriod']]

customer_df = customer_stats.merge(favorite_category, on='CustomerID', how='left')\
                            .merge(customer[['CustomerID','Region']], on='CustomerID', how='left')
                            #.merge(favorite_product, on='CustomerID', how='left')\
                            # .merge(favorite_time_period, on='CustomerID', how='left')\

print(customer_df)

    CustomerID  avg_transaction_price  num_transactions  signup_year  \
0        C0001             278.334000                 5         2022   
1        C0002             208.920000                 4         2022   
2        C0003             195.707500                 4         2024   
3        C0004             240.636250                 8         2022   
4        C0005             291.603333                 3         2022   
..         ...                    ...               ...          ...   
194      C0196             416.992500                 4         2022   
195      C0197             227.056667                 3         2023   
196      C0198             239.705000                 2         2022   
197      C0199             250.610000                 4         2022   
198      C0200             296.506000                 5         2023   

        Category         Region  
0    Electronics  South America  
1       Clothing           Asia  
2     Home Decor  South America  

In [25]:
customer_df.to_csv('Cusotmer_Info.csv',index=False)

In [26]:
customer_df_encoded = pd.get_dummies(customer_df, columns=['Category','Region'])

features = ['avg_transaction_price', 'num_transactions', 'signup_year'] + \
           [col for col in customer_df_encoded.columns if col not in ['CustomerID', 'avg_transaction_price', 'num_transactions', 'signup_year']]


In [27]:
scaler = StandardScaler()
customer_df_encoded[features] = scaler.fit_transform(customer_df_encoded[features])

cosine_sim = cosine_similarity(customer_df_encoded[features])

cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_df_encoded['CustomerID'], columns=customer_df_encoded['CustomerID'])

In [28]:
def get_top_3_lookalikes(customer_id, top_n=3):
    similar_customers = cosine_sim_df[customer_id].sort_values(ascending=False).iloc[1:top_n+1]
    return list(zip(similar_customers.index, similar_customers.values))

lookalike_map = {}
for customer_id in customer_df_encoded['CustomerID'][:20]:  
    lookalike_map[customer_id] = get_top_3_lookalikes(customer_id)

In [29]:
lookalike_df = pd.DataFrame(list(lookalike_map.items()), columns=['CustomerID', 'Lookalikes'])

lookalike_df.to_csv('Lookalike.csv', index=False)

In [30]:
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[(C0192, 0.9885334395887355), (C0184, 0.926896..."
1,C0002,"[(C0106, 0.9832049178481428), (C0134, 0.977560..."
2,C0003,"[(C0031, 0.9857350786431255), (C0052, 0.975065..."
3,C0004,"[(C0147, 0.9921528399065114), (C0165, 0.983377..."
4,C0005,"[(C0007, 0.972243044563755), (C0186, 0.9309072..."
